# Deployment Considerations

A strong point to consider deployment from the early stages of scoping a project is made. Pressing on with the development in earnest prior to understanding the architecture tha app will be hosted on can cause all sorts of problems that ultimately cost more development time. 

Once a Data Scientist presents a trained model for hosting, here are some effective steps to consider:

1. **Compatability with infrastructure.** Will it run on the target infrastructure?
2. **Reproducibility.**
    - metadata store
    - data versioning
3. **Validation.**
    - data profile to store expected value references for comparison
4. **Logging.**
    - Monitoring of performance over time.
    - logging at 3 key stages, input data, system errors or warnings, predicted values.
5. **Maintenance.**
    - Bugfixes, feature requests etc will need to be handled.
    - How confident will you be in adjusting the codebase may be related to the
    comprehensive test suite included with the code.

